<a href="https://colab.research.google.com/github/alexgabriel28/kaggle_competitions/blob/main/Kaggle_Tabular_Playground_Nov2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Upload API key file (.json)
from google.colab import files
files.upload()

#Create kaggle folder to save copy of API key file
!mkdir ~/.kaggle

#Copy API key file
!cp kaggle.json ~/.kaggle/

#Ensure readability of key file by current user only
!chmod 600 ~/.kaggle/kaggle.json

#Download dataset
!kaggle competitions download -c tabular-playground-series-nov-2021

#Uncompress dataset silently (-qq)
!unzip -qq train.csv.zip
!unzip -qq test.csv.zip

Saving kaggle.json to kaggle.json
 92% 208M/225M [00:01<00:00, 111MB/s]
100% 225M/225M [00:01<00:00, 133MB/s]
  0% 0.00/1.16M [00:00<?, ?B/s]
100% 1.16M/1.16M [00:00<00:00, 166MB/s]
 90% 182M/202M [00:01<00:00, 141MB/s]
100% 202M/202M [00:01<00:00, 149MB/s]


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import seaborn as sns

df_train = pd.read_csv("train.csv", index_col = 0)
df_train_y = df_train[["target"]]
df_train = df_train.drop(["target"], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_1_300k, X_2_300k, y_1_300k, y_2_300k = train_test_split(df_train, df_train_y, test_size = 0.5, stratify = df_train_y)

In [ ]:
!pip install deap update_checker tqdm stopit xgboost
!pip install tpot

     |████████████████████████████████| 160 kB 5.2 MB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11952 sha256=5a87035a2972fe1da2b1f1f755621aab3e19db452d10626f6d798a5f4af81858
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
     |████████████████████████████████| 87 kB 3.7 MB/s 
     |████████████████████████████████| 173.5 MB 59 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [ ]:
from tpot import TPOTClassifier
tpot = TPOTClassifier(generations = 4, population_size = 3, offspring_size = 10, scoring = "roc_auc", cv = 5, verbosity = 3, n_jobs = -1, early_stop = 3)
tpot.fit(df_train, df_train_y)
print(tpot.score(X_2_300k, y_2_300k))

32 operators have been imported by TPOT.


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimization Progress:   0%|          | 0/43 [00:00<?, ?pipeline/s]

Skipped pipeline #2 due to time out. Continuing to the next pipeline.
Skipped pipeline #6 due to time out. Continuing to the next pipeline.
Skipped pipeline #8 due to time out. Continuing to the next pipeline.
Skipped pipeline #10 due to time out. Continuing to the next pipeline.
Skipped pipeline #15 due to time out. Continuing to the next pipeline.

Generation 1 - Current Pareto front scores:

-1	0.6265331637696689	SGDClassifier(CombineDFs(input_matrix, input_matrix), SGDClassifier__alpha=0.0, SGDClassifier__eta0=0.01, SGDClassifier__fit_intercept=False, SGDClassifier__l1_ratio=0.75, SGDClassifier__learning_rate=constant, SGDClassifier__loss=perceptron, SGDClassifier__penalty=elasticnet, SGDClassifier__power_t=1.0)

-2	0.6717966524528761	SGDClassifier(MaxAbsScaler(input_matrix), SGDClassifier__alpha=0.0, SGDClassifier__eta0=0.01, SGDClassifier__fit_intercept=False, SGDClassifier__l1_ratio=0.75, SGDClassifier__learning_rate=constant, SGDClassifier__loss=perceptron, SGDClassifier__penal

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  "X does not have valid feature names, but"


0.6883484245001557


In [ ]:
test_data = pd.read_csv("test.csv", index_col = 0)
predictions = tpot.predict(test_data)

In [ ]:
!unzip -qq sample_submission.csv.zip
submission_logic = pd.read_csv("sample_submission.csv")

In [ ]:
submission_1 = pd.DataFrame(test_data.index)
submission_1["target"] = predictions

In [ ]:
submission_1.to_csv("submission_1.csv", index = False)

In [ ]:
submission_1.shape

(540000, 2)

In [ ]:
from sklearn.feature_selection import VarianceThreshold
corr = df_train.corr().abs()

sel = VarianceThreshold(threshold=1)

sel.fit(df_train / df_train.mean())
#Show True/False for the features, if selected or not with .get_support()
mask = sel.get_support()

#Use mask to reduce features
red_df_train = df[:, mask]

In [ ]:
import xgboost as xgb

xgb_clf = xgb.XGBClassifier(n_estimators = 100, max_depth = 30)
xgb_clf.fit(X_1_300k, y_1_300k)

In [ ]:
xgb_clf.score(X_2_300k, y_2_300k)

0.6614533333333333